In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy as sp

import datetime as dt

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv("supermarket_sales - Sheet1.csv")

In [3]:
df.head(4)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4


In [4]:
print(df["Branch"].value_counts())
"""
A is Yangon City,
B is Mandalay City,
C is Naypyitaw City.
Removing the City Column as it will not be beneficial for us (No extra value will be insighted here).

"""
df.drop(["City"],inplace=True, axis=1)
df.head()

Branch
A    340
B    332
C    328
Name: count, dtype: int64


,Invoice ID,Branch,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [5]:
# Invoice ID : Removing the Invoice ID as it doesn't have any significant role in the sales.
# Gender : Gender may not be directly relevant to sales forecasting for a supermarket. 
# Gross Margin Percentage : Also removing the Gross Margin Percentage as; It's a derived metric.
# Gross Income : Here in this data we have gross income == Taxv 5% so it will also not provide us the more value to better to remove it.
# Rating : Not directly related to sales forecasting objectives.
df.drop(["Invoice ID","Gender","gross margin percentage","gross income","Rating"],inplace=True, axis=1)
df.head()

,Branch,Customer type,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs
0,A,Member,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83
1,C,Normal,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40
2,A,Normal,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31
3,A,Member,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76
4,A,Normal,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17


In [6]:
df = df[["Date","Time","Branch","Customer type","Product line", "Unit price","Quantity", "Tax 5%","cogs","Total","Payment"]]
df.head()

,Date,Time,Branch,Customer type,Product line,Unit price,Quantity,Tax 5%,cogs,Total,Payment
0,1/5/2019,13:08,A,Member,Health and beauty,74.69,7,26.1415,522.83,548.9715,Ewallet
1,3/8/2019,10:29,C,Normal,Electronic accessories,15.28,5,3.8200,76.40,80.2200,Cash
2,3/3/2019,13:23,A,Normal,Home and lifestyle,46.33,7,16.2155,324.31,340.5255,Credit card
3,1/27/2019,20:33,A,Member,Health and beauty,58.22,8,23.2880,465.76,489.0480,Ewallet
4,2/8/2019,10:37,A,Normal,Sports and travel,86.31,7,30.2085,604.17,634.3785,Ewallet


In [7]:
#Dropping Customer type, Payment time and Product line
df.drop(["Payment","Customer type","Product line"], inplace=True, axis=1)

In [8]:
df.head()

,Date,Time,Branch,Unit price,Quantity,Tax 5%,cogs,Total
0,1/5/2019,13:08,A,74.69,7,26.1415,522.83,548.9715
1,3/8/2019,10:29,C,15.28,5,3.8200,76.40,80.2200
2,3/3/2019,13:23,A,46.33,7,16.2155,324.31,340.5255
3,1/27/2019,20:33,A,58.22,8,23.2880,465.76,489.0480
4,2/8/2019,10:37,A,86.31,7,30.2085,604.17,634.3785


In [9]:
df["Date"]= pd.to_datetime(df["Date"])
df["Time"]= pd.to_datetime(df["Time"],format="%H:%M")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1000 non-null   datetime64[ns]
 1   Time        1000 non-null   datetime64[ns]
 2   Branch      1000 non-null   object        
 3   Unit price  1000 non-null   float64       
 4   Quantity    1000 non-null   int64         
 5   Tax 5%      1000 non-null   float64       
 6   cogs        1000 non-null   float64       
 7   Total       1000 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(1)
memory usage: 62.6+ KB


In [10]:
morning_start = pd.to_datetime('04:00:00').time()
afternoon_start = pd.to_datetime('12:00:00').time()
night_start = pd.to_datetime('18:00:00').time()

# Defining a function to determine the time phase
def get_time_phase(time):
    if time >= morning_start and time < afternoon_start:
        return 'Morning'
    elif time >= afternoon_start and time < night_start:
        return 'Afternoon'
    else:
        return 'Night'

# Apply the function to create the new column
df['Time Phase'] = df['Time'].dt.time.apply(get_time_phase)
df.drop(["Time"],inplace=True, axis=1)
df.head()

,Date,Branch,Unit price,Quantity,Tax 5%,cogs,Total,Time Phase
0,2019-01-05,A,74.69,7,26.1415,522.83,548.9715,Afternoon
1,2019-03-08,C,15.28,5,3.8200,76.40,80.2200,Morning
2,2019-03-03,A,46.33,7,16.2155,324.31,340.5255,Afternoon
3,2019-01-27,A,58.22,8,23.2880,465.76,489.0480,Night
4,2019-02-08,A,86.31,7,30.2085,604.17,634.3785,Morning


In [11]:
df.sort_values(ascending=True,inplace=True,by="Date",ignore_index=True)

In [12]:
df.drop(["Branch"],inplace=True, axis =1 )
df.head()

,Date,Unit price,Quantity,Tax 5%,cogs,Total,Time Phase
0,2019-01-01,21.12,8,8.448,168.96,177.408,Night
1,2019-01-01,65.74,9,29.583,591.66,621.243,Afternoon
2,2019-01-01,47.59,8,19.036,380.72,399.756,Afternoon
3,2019-01-01,27.04,4,5.408,108.16,113.568,Night
4,2019-01-01,62.87,2,6.287,125.74,132.027,Morning


In [13]:
df.groupby(["Date","Time Phase"], group_keys=True)[['Quantity','cogs',"Tax 5%"]].apply(lambda x: x)

Quantity    cogs  Tax 5%
Date       Time Phase                              
2019-01-01 Afternoon  1           9  591.66  29.583
                      2           8  380.72  19.036
                      5          10  742.20  37.110
                      6           2  126.44   6.322
           Morning    4           2  125.74   6.287
...                             ...     ...     ...
2019-03-30 Afternoon  996         8   81.44   4.072
                      999         5  322.20  16.110
           Morning    998         5  447.40  22.370
           Night      990         8  580.16  29.008
                      997         3  290.46  14.523

[1000 rows x 3 columns]

In [14]:
main_df = df.groupby(["Date","Time Phase"], group_keys=True)[["Quantity","cogs","Tax 5%","Total"]].sum().reset_index()

In [15]:
df.head()

,Date,Unit price,Quantity,Tax 5%,cogs,Total,Time Phase
0,2019-01-01,21.12,8,8.448,168.96,177.408,Night
1,2019-01-01,65.74,9,29.583,591.66,621.243,Afternoon
2,2019-01-01,47.59,8,19.036,380.72,399.756,Afternoon
3,2019-01-01,27.04,4,5.408,108.16,113.568,Night
4,2019-01-01,62.87,2,6.287,125.74,132.027,Morning


In [16]:
main_df.head()

,Date,Time Phase,Quantity,cogs,Tax 5%,Total
0,2019-01-01,Afternoon,29,1841.02,92.051,1933.071
1,2019-01-01,Morning,24,1583.02,79.151,1662.171
2,2019-01-01,Night,28,1095.18,54.759,1149.939
3,2019-01-02,Afternoon,34,1253.38,62.669,1316.049
4,2019-01-02,Night,14,599.48,29.974,629.454


In [17]:
main_df["Month name"] = main_df["Date"].dt.month_name()
main_df["Day name"] = main_df["Date"].dt.day_name()
main_df["Weekend"] = np.where(main_df["Day name"].isin(["Saturday","Sunday"]),1,0) #Making 1 is it is weekend else 0
main_df["Date"] = main_df["Date"].dt.day

In [18]:
main_df.head()

,Date,Time Phase,Quantity,cogs,Tax 5%,Total,Month name,Day name,Weekend
0,1,Afternoon,29,1841.02,92.051,1933.071,January,Tuesday,0
1,1,Morning,24,1583.02,79.151,1662.171,January,Tuesday,0
2,1,Night,28,1095.18,54.759,1149.939,January,Tuesday,0
3,2,Afternoon,34,1253.38,62.669,1316.049,January,Wednesday,0
4,2,Night,14,599.48,29.974,629.454,January,Wednesday,0


In [19]:
print(main_df["Month name"].value_counts())

Month name
January     88
March       85
February    79
Name: count, dtype: int64


In [20]:
#Transforming the Months Name using the Ordinal Encoder
"""
0 for January
1 for February
2 for March

"""
oe_month = OrdinalEncoder(categories=[["January","February","March"]])
main_df["Month name"] = oe_month.fit_transform(main_df[["Month name"]])
main_df["Month name"] += 1
""" Now : 1 for January, 2 for February, 3 for March """


' Now : 1 for January, 2 for February, 3 for March '

In [21]:
# Encoding the week names using ordinal encoder
oe_Dayname = OrdinalEncoder(categories=[['Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']])
main_df['Day name'] = oe_Dayname.fit_transform(main_df[["Day name"]])
main_df["Day name"] += 1

In [22]:
main_df.head()

,Date,Time Phase,Quantity,cogs,Tax 5%,Total,Month name,Day name,Weekend
0,1,Afternoon,29,1841.02,92.051,1933.071,1.0,2.0,0
1,1,Morning,24,1583.02,79.151,1662.171,1.0,2.0,0
2,1,Night,28,1095.18,54.759,1149.939,1.0,2.0,0
3,2,Afternoon,34,1253.38,62.669,1316.049,1.0,3.0,0
4,2,Night,14,599.48,29.974,629.454,1.0,3.0,0


In [23]:
oe_timephase = OrdinalEncoder(categories=[["Morning","Afternoon","Night"]])
main_df["Time Phase"] = oe_timephase.fit_transform(main_df[["Time Phase"]])
main_df["Time Phase"] += 1
main_df.head()

,Date,Time Phase,Quantity,cogs,Tax 5%,Total,Month name,Day name,Weekend
0,1,2.0,29,1841.02,92.051,1933.071,1.0,2.0,0
1,1,1.0,24,1583.02,79.151,1662.171,1.0,2.0,0
2,1,3.0,28,1095.18,54.759,1149.939,1.0,2.0,0
3,2,2.0,34,1253.38,62.669,1316.049,1.0,3.0,0
4,2,3.0,14,599.48,29.974,629.454,1.0,3.0,0


In [24]:
main_df = main_df[["Date","Month name","Day name","Time Phase","Weekend","Quantity", "cogs", "Tax 5%", "Total"]]

In [25]:
main_df.head()

,Date,Month name,Day name,Time Phase,Weekend,Quantity,cogs,Tax 5%,Total
0,1,1.0,2.0,2.0,0,29,1841.02,92.051,1933.071
1,1,1.0,2.0,1.0,0,24,1583.02,79.151,1662.171
2,1,1.0,2.0,3.0,0,28,1095.18,54.759,1149.939
3,2,1.0,3.0,2.0,0,34,1253.38,62.669,1316.049
4,2,1.0,3.0,3.0,0,14,599.48,29.974,629.454


In [28]:
# main_df.to_csv("Supermarket-pred-data.csv")
main_df.to_excel("Supermarket-pred-data.xlsx")